In [ ]:
#assign ID to notebook (to be replaced by nbparameterise)
id = "AAGD02"

In [ ]:
#Import libraries
from flask import request
from ipy_table import *
import os
from IPython.display import HTML, display, clear_output, Markdown, Javascript
import IPython.core.display as di
import IPython.display as display
import subprocess
import urllib.request, json, urllib.error
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import sys
from utils import *
import datetime
import time
import pandas as pd
import IPython
import IPython.core.display as di
from IPython.display import display
from IPython.display import Markdown as md
from IPython.display import Image 
import os
import requests
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas.io.json import json_normalize
import tabulate
import sys
from nbparameterise import extract_parameters, parameter_values, replace_definitions
import nbformat


#set some display style
display(HTML("<style>.container { width:100% !important; }</style>"))
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)
pd.options.display.max_rows = 4000

In [ ]:
%%javascript
//Javascript for creating the tabs in the resulting HTML Docs
function openPage(pageName, elmnt, color) {
  // Hide all elements with class="tabcontent" by default */
  var i, tabcontent, tablinks;
  tabcontent = document.getElementsByClassName("tabcontent");
  for (i = 0; i < tabcontent.length; i++) {
    tabcontent[i].style.display = "none";
  }

  // Remove the background color of all tablinks/buttons
  tablinks = document.getElementsByClassName("tablink");
  for (i = 0; i < tablinks.length; i++) {
    tablinks[i].style.backgroundColor = "";
  }

  // Show the specific tab content
  document.getElementById(pageName).style.display = "block";

  // Add the specific color to the button used to open the tab content
  elmnt.style.backgroundColor = color;
}


In [ ]:
#Return metadata summary of sample in question
with urllib.request.urlopen("https://blobtoolkit.genomehubs.org/api/v1/summary/" + id) as sumUrl:
    data = json.loads(sumUrl.read().decode())
    df = json_normalize(data['summaryStats'],sep="_")
    

    


#Define image filenames
circle = open('img/' + id + '_Circle.png','wb')
hex = open('img/' + id + '_Hex.png','wb')
square = open('img/' + id + '_Square.png','wb')
cumulative = open('img/' + id + '_Cumulative.png','wb')
snail = open('img/' + id + '_Snail.png','wb')

#Define HTML requests for each image & check whether they exist
summaryReq = Request('https://blobtoolkit.genomehubs.org/api/v1/summary/' + id)
circleReq = Request('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/blob/circle')
hexReq = Request('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/blob/hex')
squareReq = Request('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/blob/square')
cumulaReq = Request('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/cumulative')
snailReq = Request('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/snail')
summaryReq = Request('https://blobtoolkit.genomehubs.org/api/v1/summary/'+ id)
try:
    #If they exist....
    circResponse = urlopen(circleReq)
    hexResponse = urlopen(hexReq)
    squareResponse = urlopen(squareReq)
    cumulaResponse = urlopen(cumulaReq)
    snailResponse = urlopen(snailReq)
except HTTPError as e:
    pass
except URLError as e:
    pass
else:
    #...download them
    circle.write(urllib.request.urlopen('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/blob/circle').read())
    hex.write(urllib.request.urlopen('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/blob/hex').read())
    square.write(urllib.request.urlopen('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/blob/square').read())
    cumulative.write(urllib.request.urlopen('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/cumulative').read())
    snail.write(urllib.request.urlopen('https://blobtoolkit.genomehubs.org/api/v1/image/' + id + '/snail').read())
    hex.close()
    square.close()
    cumulative.close()
    snail.close()

    #Define taxonomy table lists
    taxValues = []
    taxHeaders = []
    lineageList = []

    #Define taxonomy columns & values
    taxCols = df.loc[:, df.columns.str.startswith('taxonomy_')]
    for col in taxCols:
        colpos = df.columns.get_loc(col)
        colname = df.columns[colpos]
        taxValues.append(taxCols[col].item())
        taxHeaders.append(colname)

    #Assign lineage to string
    blobLine = taxValues[0]

    #Remove lineage
    #if (len(taxValues) >= 3):
    del taxValues[0]
    del taxHeaders[0]
   # else:
    #    pass

    #Define lists for base composition table
    baseTable = []
    baseValues = []
    baseHeaders = []

    #Define base composition columns & values
    baseCols = df.loc[:, df.columns.str.startswith('baseComposition_')]
    for col in baseCols:
        colpos = df.columns.get_loc(col)
        colname = df.columns[colpos]
        if "_at" in colname:
            colname = colname.replace("baseComposition_at","AT")
        if "_gc" in colname:
            colname = colname.replace("baseComposition_gc","GC")
        if "_n" in colname:
            colname = colname.replace("baseComposition_n","N")
        baseValues.append(baseCols[col].item())
        baseHeaders.append(colname)
    baseValues[0] = baseValues[0]*100
    baseValues[0] = '%.2f' % baseValues[0] + "%"
    baseValues[1] = baseValues[1]*100
    baseValues[1] = '%.2f' % baseValues[1] + "%"
    baseValues[2] = baseValues[2]*100
    baseValues[2] = '%.2f' % baseValues[2]  + "%"

    #Base composition table
    baseTable.insert(0,baseHeaders)
    baseTable.insert(1,baseValues)
    ipybaseTable = make_table(baseTable)
    ipybaseTable.apply_theme('basic')
    set_global_style(float_format='%0.3E')
    set_cell_style(0, 1, align='center')
    set_row_style(0,color='MediumOrchid')
    baseTableHTML = render()._repr_html_()
    baseTableHTML = baseTableHTML.replace("<b>","<b style=\"color:white\"><font size=\"3\">")

    #Define lists for assembly statistics table
    statValues = []
    statTable = []
    statHeaders = []
    statValuesPercent = []

    statCols = df.loc[:, df.columns.str.startswith('stats_')]
    for col in statCols:
        colpos = df.columns.get_loc(col)
        colname = df.columns[colpos]
        if "_noHit" in colname:
            colname = colname.replace("stats_noHit","No Hit")
        statValVar = statCols[col].item()*100
        statValVar = '%.2f' % statValVar
        statValVar = statValVar + "%"
        statValues.append(statValVar)
        statHeaders.append(colname)

    #Multiply stat values by 100 for percentages    
    #Remove span/n50
    del statValues[1]
    del statHeaders[1]
    del statHeaders[1]

    #Stat & Target table
    statHeaders.append(str(taxValues[1].capitalize() + " " + taxValues[0]))
    statTable.insert(0,statHeaders)
    statTable.insert(1,statValues)
    ipystatTable = make_table(statTable)
    ipystatTable.apply_theme('basic')
    set_global_style(float_format='%0.3E')
    set_cell_style(0, 1, align='center')
    set_cell_style(1, 1, align='center')
    set_global_style(color='white')
    set_row_style(0,color='MediumOrchid')
    statTableHTML = render()._repr_html_()

    statTableHTML = statTableHTML.replace("<b>","<b style=\"color:white\"><font size=\"3\">")
    statTableHTML = statTableHTML.replace("</b>","</b>")
    

    #Display lineage
    blobLine = "<h2 align=\"left\">Lineage: " + blobLine + " </h2>" 

    #Declare Lists for Hits table
    hitTable = []
    hitPhylumList = []
    hitCountList = []
    hitn50List = []
    hitSpanList = []
    totalTable = []

    #Return count columns & values & lineage(s)
    hitCountCols = df.loc[:, df.columns.str.endswith('_count')]
    for col in hitCountCols:
        hitCountComma = format(hitCountCols[col][0],",d")
        hitCountList.append(hitCountComma)
        header = col.replace("hits_","")
        header = header.replace("_count","")
        hitPhylumList.append(header)

    #Return n50 columns & values
    hitn50Cols = df.loc[:, df.columns.str.endswith('_n50')]
    for col in hitn50Cols:
        hitn50Comma = format(hitn50Cols[col][0],",d")
        hitn50List.append(hitn50Comma)

    #Return span columns & values
    hitSpanCols = df.loc[:, df.columns.str.endswith('_span')]
    for col in hitSpanCols:
        hitSpanComma = format(hitSpanCols[col][0],",d")
        hitSpanList.append(hitSpanComma)

    #Insert values into list
    for item in range(len(hitSpanList)):
        rowList = [hitCountList[item],hitn50List[item],hitSpanList[item],hitPhylumList[item]]
        hitTable.insert(item+1,rowList)

    #Make total table
    totalValues = [hitTable[-1][0], hitTable[-1][1], hitTable[-1][2], baseValues[0], baseValues[1], baseValues[2]]
    totalTableHeaders = ['Count','n50','Span','AT', 'GC', 'N']
    totalTable.insert(0,totalTableHeaders)
    totalTable.insert(1,totalValues)
    ipyTotalTable = make_table(totalTable)
    ipyTotalTable.apply_theme('basic')
    set_global_style(float_format='%0.3E')
    set_cell_style(0, 1, align='center')
    set_cell_style(0, 2, align='center')
    set_cell_style(0, 3, align='center')
    set_global_style(color='white')
    set_row_style(0,color='MediumOrchid')

    totalTableHTML = render()._repr_html_()
    totalTableHTML = totalTableHTML.replace("<b>","<b style=\"color:white\"><font size=\"3\">")

    #Declare lists for BUSCO table
    buscStrList = []
    buscLineList = []
    buscTotalList = []
    rowList = []
    buscTable = []


    #Return BUSCO string columns & return string & lineage values
    strCols = df.loc[:, df.columns.str.endswith('_string')]
    for col in strCols:
        buscStrList.append(strCols[col][0])
        header = col.replace("busco_","")
        header = header.replace("_odb9_string","")
        buscLineList.append(header)

    #Insert values into list
    for item in range(len(buscLineList)):
        rowList = [buscLineList[item],buscStrList[item]]
        buscTable.insert(item+1,rowList)

    #Create BUSCO table
    buscTableHeaders = ['Lineage','String']
    buscTable.insert(0,buscTableHeaders) 
    ipyBuscTable = make_table(buscTable)
    ipyBuscTable.apply_theme('basic')
    set_global_style(float_format='%0.3E')
    set_cell_style(0, 1, align='center')
    set_row_style(0,color='DarkOrchid')
    set_global_style(color='white')
    set_row_style(0,color='MediumOrchid')

    buscTableHTML = render()._repr_html_()
    buscTableHTML = buscTableHTML.replace("<b>","<b style=\"color:white\"><font size=\"3\">")

    #Create HIT table
    del hitTable[-1]
    hitTableHeaders = ['Count','n50','Span','Phylum']
    hitTable.insert(0,hitTableHeaders)
    ipyHitTable = make_table(hitTable)
    ipyHitTable.apply_theme('basic')
    set_global_style(float_format='%0.3E')
    set_cell_style(0, 1, align='center')
    set_cell_style(0, 2, align='center')
    set_cell_style(0, 3, align='center')
    set_global_style(color='white')
    set_row_style(0,color='MediumOrchid')

    hitTableHTML = render()._repr_html_()
    hitTableHTML = hitTableHTML.replace("<b>","<b style=\"color:white\"><font size=\"3\">")

    #Display/set up the HTML Doc webpage as desired by calling variables assigned to HTML outputs & add some CSS
    hyperLink = "<a href=\"https://blobtoolkit.genomehubs.org/view/" + id + "/dataset/" + id + "/blob#Filters \"target=\"_blank\"><center>Click Here for Interactive BTK Viewer Analysis (New Tab)</center></a>"
    blobPlotHeader = "<h2 align=\"center\">Blobplot (GC vs Coverage by Phylum)</h2>"
    blobPlotHTML = "<table><tr><td><img src='img/" + id + "_Circle.png' alt=\"Drawing\" style=\"border:2px solid purple\"></td></tr></table>"
    noteBookHeader = """ 
    <div class="header">
      <h1>BlobToolKit Assembly Analysis (""" + id + """) </h1>
    </div> 

    <style>.header {
      padding: 60px;
      text-align: center;
      background:#31323f;
      color: white;
      font-size: 30px;
    }</style>
    """
    buscHeader = "<h2 align=\"left\">BUSCO Strings</h2>"
    hitsHeader = "<h2 align=\"left\">Hits</h2>"
    assemblyStatsHeader = "<h2 align=\"left\">Percentage of Sequences with Hits in Public Databases (nt & Uniprot)</h2>"
    totalHeader = "<h2 align=\"left\">Assembly Statistics Summary</h2>"
    snailAndCumHeader = "<h2><center>Snail Plot of Summary Statistics (left) & Cumulative Length Plot of Scaffolds by Phylum (right)</center></h2>"
    snailAndCumHTML = "<table align='center'><tr><td><img src='img/" + id + "_Snail.png' alt=\"Drawing\" style=\"border:2px solid purple\"></td><td><img src='img/" + id + "_Cumulative.png' alt=\"Drawing\" style=\"border:2px solid purple\"></td></tr></table>"
    display(HTML("""
    <button class="tablink" onclick="openPage('GC vs Coverage by Phylum', this)" id="defaultOpen">GC vs Coverage by Phylum (Default)</button>
    <button class="tablink" onclick="openPage('Summary Tables', this)">Summary Tables</button>
    <button class="tablink" onclick="openPage('Snail Plot & Cumulative Plot', this)">Snail Plot & Cumulative Plot</button>
    <br><br>


    """ + noteBookHeader + "<br>"+ hyperLink +"""

    <div id="GC vs Coverage by Phylum" class="tabcontent">
      """+ blobPlotHeader + blobPlotHTML + blobLine + """
    </div>

    <div id="Summary Tables" class="tabcontent">
      """ + assemblyStatsHeader + statTableHTML + totalHeader + totalTableHTML + buscHeader + buscTableHTML + hitsHeader + hitTableHTML +"""  
    </div>

    <div id="Snail Plot & Cumulative Plot" class="tabcontent">
      """ + snailAndCumHeader + snailAndCumHTML +"""
    </div>


    <style>
    /* Set height of body and the document to 100% to enable "full page tabs" */
    body, html {
      height: 100%;
      margin: 0;
      font-family: Arial;
    }

    /* Style tab links */
    .tablink {
      background-color: #31323f;
      color: white;
      float: left;
      border: none;
      outline: none;
      cursor: pointer;
      padding: 14px 16px;
      font-size: 17px;
      width: 25%;
    }

    .tablink:hover {
      background-color: #9c528b;
    }

    /* Style the tab content (and add height:100% for full page content) */
    .tabcontent {
      color: black;
      display: none;
      padding: 100px 20px;
      height: 100%;
    }

    div.container {
      max-width: 1250px;
    }

    div.output_html, div.rendered_html, div.output_subarea {
      overflow: visible;
      max-width: none;
      margin-top: -39px;
    }

    button.tablink {
      border: solid white 1px;
      border-bottom: solid white 2px;
      padding: 9px 16px;
      font-size: 15px;
      width: 33.33%;
    }

    document.getElementById("defaultOpen").click();

    </style>
    """))
    Javascript("Jupyter.notebook.execute_cells([4])")

In [ ]:
Javascript("document.getElementById(\"defaultOpen\").click();")